# Retrival

In [5]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
from langchain.schema import Document
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch

In [6]:
load_dotenv()
mongo_token = os.getenv("MONGODB_URL")
MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"

In [7]:
model = SentenceTransformer(MODEL_ID)

In [ ]:
# embeddings = HuggingFaceEmbeddings(model_name=MODEL_ID, model_kwargs={"device": "cpu"})

In [8]:
try:
    client = MongoClient(mongo_token)
    db = client['Tour']
    collection = db['spots']
except Exception as e:
    print(f"Error connecting to MongoDB: {e}")
    exit(1)

In [12]:
def initialize_local_llm():
    try:
        model_name = "distilgpt2"  # A smaller model suitable for CPU
        
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)
        
        pipe = pipeline(
            "text-generation",
            model=model, 
            tokenizer=tokenizer, 
            max_length=512,
            temperature=0.7,
            top_p=0.95,
            repetition_penalty=1.15,
            device=-1  # This tells it to use CPU
        )
        
        local_llm = HuggingFacePipeline(pipeline=pipe)
        return local_llm
    except Exception as e:
        print(f"Error initializing LLM: {e}")
        exit(1)

llm = initialize_local_llm()


/home/unk/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [13]:
def query_encoder(query):
    return model.encode([query])

In [14]:
def setup_chatbot(vectorstore):
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return qa_chain


In [15]:
def create_vector_store(documents):
    docs = [
        Document(
            page_content=f"{doc['name']}: {doc['description']}",
            metadata={"name": doc['name']}
        ) for doc in documents
    ]
    
    embeddings = model.encode([doc.page_content for doc in docs])
    vector_store = FAISS.from_embeddings(embeddings, [doc.page_content for doc in docs])
    return vector_store

In [16]:
def load_documents_from_mongodb():
    try:
        return list(collection.find())
    except Exception as e:
        print(f"Error loading documents from MongoDB: {e}")
        return []


In [17]:
documents = load_documents_from_mongodb()
vector_store = create_vector_store(documents)
chatbot = setup_chatbot(vector_store)

ValueError: too many values to unpack (expected 2)

In [ ]:
query = "Tell me about the best spots in Karachi"
query_embedding = query_encoder(query)

print("\nChatbot Response:")
try:
    response = chatbot({"question": query})
    print(f"Question: {query}")
    print(f"Answer: {response['answer']}")
except Exception as e:
    print(f"Error processing query: {e}")

# Cleanup
client.close()

NameError: name 'vector_search' is not defined